# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import json
import logging
from collections import namedtuple
from pathlib import Path
from typing import *

import fire
import reader
import rich
from bs4 import BeautifulSoup
from fastcore.basics import patch_to
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, "html.parser")
    return s.text

In [ ]:
# | export


class Feed:
    """RSS feed class"""

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)

In [ ]:
# | export


class PyNewsReader:
    def __init__(self, feeds=List[Feed]):
        self._dbfolder = Path().home() / ".cache/pynewsreader"
        if not self._dbfolder.exists():
            self._dbfolder.mkdir(parents=True)

        # If custom feed names exist, read them in
        feed_names_path = self._dbfolder / "feed_names.json"
        if feed_names_path.exists():
            with open(self._dbfolder / "feed_names.json", "rt") as myfile:
                self._feed_names = json.load(myfile)
        else:
            self._feed_names = {}

        # If title blacklist exists, read it in
        title_blacklist_path = self._dbfolder / "title_blacklist.json"
        if title_blacklist_path.exists():
            with open(self._dbfolder / "title_blacklist.json", "rt") as myfile:
                self._title_blacklist = json.load(myfile)
        else:
            self._title_blacklist = []

        self._reader = reader.make_reader(
            str(self._dbfolder / "db.sqlite"),
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        # If title whitelist exists, read it in
        title_whitelist_path = self._dbfolder / "title_whitelist.json"
        if title_whitelist_path.exists():
            with open(self._dbfolder / "title_whitelist.json", "rt") as myfile:
                self._title_whitelist = json.load(myfile)
        else:
            self._title_whitelist = []

        self._reader = reader.make_reader(
            str(self._dbfolder / "db.sqlite"),
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        self._reader.enable_search()

        def hook(session, request, **kwargs):
            request.headers.setdefault(
                "User-Agent",
                "Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0",
            )

        self._reader._parser.session_factory.request_hooks.append(hook)

    def _print_entries(
        self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10
    ):
        """Pretty print entries - supports reader.Reader.get_entries arguments"""
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = "Date: " + e.published.isoformat()[:10]
                else:
                    published_date = "Date: Unknown"
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = f"[bold]{self._get_feed_title(e.original_feed_url)}[/bold]"

                if e.important:
                    panel_body = ":exclamation_mark:"
                else:
                    panel_body = ""
                panel_body += f"Title: [bold]{e.title}[/bold]" + "\n"
                panel_body += str(published_date) + "\n\n"
                try:
                    panel_body += strip_html(e.summary).strip() + "\n"
                except TypeError:
                    panel_body += str(e.summary) + "\n"

                console.print(
                    Panel(
                        panel_body,
                        title=feed_title,
                        subtitle=f"[link={e.link}]{e.link}[/link]",
                    )
                )
                console.print()
            if len(displayed_links) == limit:
                return

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed"""
        if url in self._feed_names and self._feed_names[url] is not None:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def _get_entries(
        self, important: bool = None, read: Union[None, bool] = None, limit: int = 10
    ) -> List[reader.Entry]:
        """Get entries in reader.Entry format"""
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def _get_tags(self, entry: reader.Entry):
        """Get tags for a given entry"""
        return [i[0] for i in list(self._reader.get_tags(entry))]

    def _mark_matching_entries_as_read(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)
                if filter_string in i.link:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)

    def _mark_matching_entries_as_important(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as important: {i.title}")
                    self._reader.mark_entry_as_important(i)

    def _add_tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Add tag to entry"""
        reader.Reader.set_tag(entry, tag_key, tag_value)

    def _remove_tag(self, entry: reader.Entry, tag_key: str):
        """Remove tag from entry"""
        self._reader.delete_tag(entry, tag_key)

    def _mark_important(self, feed_url: str, entry_id: str):
        article = self._reader.get_entry((feed_url, entry_id))
        self._reader.mark_entry_as_important(article)

    def _mark_unimportant(self, feed_url: str, entry_id: str):
        article = self._reader.get_entry((feed_url, entry_id))
        self._reader.mark_entry_as_unimportant(article)

In [ ]:
# | export


@patch_to(PyNewsReader)
def blacklist_add(self, blacklist_string: str):
    """Add entry to blacklist"""
    if blacklist_string not in self._title_blacklist:
        self._title_blacklist.append(blacklist_string)
        with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
            json.dump(self._title_blacklist, myfile)


@patch_to(PyNewsReader)
def blacklist_remove(self, blacklist_string: str):
    """Remove entry from blacklist"""
    if blacklist_string in self._title_blacklist:
        self._title_blacklist.remove(blacklist_string)
        with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
            json.dump(self._title_blacklist, myfile)


@patch_to(PyNewsReader)
def blacklist_show(self):
    """Show blacklist"""
    print(self._title_blacklist)

In [ ]:
#| export


@patch_to(PyNewsReader)
def whitelist_add(self, whitelist_string: str):
    """Add entry to whitelist"""
    if whitelist_string not in self._title_whitelist:
        self._title_whitelist.append(whitelist_string)
        with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
            json.dump(self._title_whitelist, myfile)
        for entry in self._get_entries(limit=None):
            if whitelist_string in entry.title:
                self._reader.mark_entry_as_important(entry)


@patch_to(PyNewsReader)
def whitelist_remove(self, whitelist_string: str):
    """Remove whitelist entry"""
    if whitelist_string in self._title_whitelist:
        self._title_whitelist.remove(whitelist_string)
        with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
            json.dump(self._title_whitelist, myfile)
        for entry in self._get_entries(limit=None):
            if whitelist_string in entry.title:
                self._reader.mark_entry_as_unimportant(entry)


@patch_to(PyNewsReader)
def whitelist_show(self):
    """Show whitelist"""
    print(self._title_whitelist)

In [ ]:
# | export


@patch_to(PyNewsReader)
def update(self, workers: int = 8):
    """Update feeds and search"""
    self._reader.update_feeds(workers=workers)
    self._reader.update_search()
    if len(self._title_blacklist) > 0:
        self._mark_matching_entries_as_read(self._title_blacklist)
    if len(self._title_whitelist) > 0:
        self._mark_matching_entries_as_important(self._title_whitelist)

In [ ]:
# | export


@patch_to(PyNewsReader)
def add_feed(self, feed: Union[Feed, str]):
    """Add feed to pynewsreader

    Args:
        feed (Feed): pynewsreader Feed to add
    """
    if isinstance(feed, Feed):
        self._feed_names[feed.url] = feed.name
        self._reader.add_feed(feed.url.rstrip("/"), exist_ok=True)
    elif isinstance(feed, str):
        self._reader.add_feed(feed.rstrip("/"), exist_ok=True)
    else:
        raise Exception("Must be str or Feed type to add")

    # Save names to file
    with open(self._dbfolder / "feed_names.json", "wt") as myfile:
        json.dump(self._feed_names, myfile)


@patch_to(PyNewsReader)
def remove_feed(self, feed: Union[Feed, str]):
    """Remove feed from pynewsreader instance

    Args:
        feed (Union[Feed, str]): Feed to remove
    """

    if isinstance(feed, Feed):
        self._reader.delete_feed(feed.url)
    elif isinstance(feed, str):
        self._reader.delete_feed(feed)
    else:
        raise Exception(TypeError)


@patch_to(PyNewsReader)
def feeds(self):
    """List pynewsreader feeds

    Returns:
        List[str]: List of names of current pynewsreader feeds
    """
    feed_object = namedtuple("Feeds", ["url", "name"])
    return [
        feed_object(i.url, self._get_feed_title(i.url))
        for i in self._reader.get_feeds()
    ]

In [ ]:
# | export


@patch_to(PyNewsReader)
def show(
    self,
    limit: int = 6,
    read: bool = False,
    important: bool = None,
    mark_as_read: bool = True,
):
    """Pretty print entries

    Args:
        limit (int, optional): Number of entries to show. Defaults to 5.
        read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
        mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
    """
    self._print_entries(
        self._get_entries(read=read, important=important, limit=limit * 2),
        limit=limit,
        mark_as_read=mark_as_read,
    )

In [ ]:
# | export


@patch_to(PyNewsReader)
def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
    """Search entries and pretty print results

    Args:a
        query (str): Search query
        mark_as_read (bool, optional): Mark results as read? Defaults to True.
    """
    entries = [self._reader.get_entry(i) for i in self._reader.search_entries(query)]
    if len(entries) > 0:
        self._print_entries(
            entries,
            mark_as_read=mark_as_read,
            limit=limit,
        )

In [ ]:
r = PyNewsReader()

## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url="https://ricochet.media/en/feed", name="Richochet Media"),
    Feed(url="https://thetyee.ca/rss2.xml", name="The Tyee"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://www.theguardian.com/science/rss",
    "https://www.theguardian.com/world/canada/rss",
]:
    r.add_feed(Feed(i))

## Update Feeds

In [ ]:
r.update()

Marking entry as important: Harold the Mortgage Closer accused of charging ‘vulnerable’ people interest rates up to
76 per cent

## List Feeds

In [ ]:
r.feeds()

[Feeds(url='https://openai.com/blog/rss.xml', name='OpenAI Blog'),
 Feeds(url='https://ricochet.media/en/feed', name='Richochet Media'),
 Feeds(url='https://www.macleans.ca/feed', name='https://www.macleans.ca/feed'),
 Feeds(url='https://blog.google/technology/ai/rss', name='AI'),
 Feeds(url='https://www.answer.ai/index.xml', name='Answer.ai Blog'),
 Feeds(url='https://www.theguardian.com/world/canada/rss', name='Canada | The Guardian'),
 Feeds(url='https://www.cbc.ca/webfeed/rss/rss-business', name='CBC | Business'),
 Feeds(url='https://www.cbc.ca/webfeed/rss/rss-canada', name='CBC | Canada'),
 Feeds(url='https://www.cbc.ca/webfeed/rss/rss-canada-ottawa', name='CBC | Ottawa'),
 Feeds(url='https://www.cbc.ca/webfeed/rss/rss-technology', name='CBC | Technology'),
 Feeds(url='https://www.cbc.ca/webfeed/rss/rss-topstories', name='CBC | Top Stories'),
 Feeds(url='https://www.cbc.ca/webfeed/rss/rss-world', name='CBC | World'),
 Feeds(url='https://engineering.fb.com/feed', name='Facebook AI 

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(limit=5, read=None, mark_as_read=False)

╭─────────────────────────────────────────────── CBC | Top Stories ───────────────────────────────────────────────╮
│ Title: Feds announce forum on antisemitism following incidents at Jewish school and synagogue                   │
│ Date: 2024-12-20                                                                                                │
│                                                                                                                 │
│ The federal government will hold a national forum on combatting antisemitism in February — news that comes as   │
│ police investigate a suspicious fire at a Montreal synagogue and gunshots aimed at a Toronto Jewish girls       │
│ school.                                                                                                         │
│                                                                                                                 │
╰─ ]8;id=750559;https://www.cbc.ca/news/politics/feds-announce-forum-on-antisemitism-following-incidents-at-jewish-school-and-synagogue-1.7416928?cmp=rss\https://www.cbc.ca/news/politics/feds-announce-forum-on-antisemitism-following-incidents-at-jewish-school-and-]8;;\─╯

╭──────────────────────────────────────────── Simon Willison's Weblog ────────────────────────────────────────────╮
│ Title: OpenAI o3 breakthrough high score on ARC-AGI-PUB                                                         │
│ Date: 2024-12-20                                                                                                │
│                                                                                                                 │
│ OpenAI o3 breakthrough high score on ARC-AGI-PUB                                                                │
│ François Chollet is the co-founder of the ARC Prize and had advanced access to today's o3 results. His article  │
│ here is the most insightful coverage I've seen of o3, going beyond just the benchmark results to talk about     │
│ what this all means for the field in general.                                                                   │
│ One fascinating detail: it cost $6,677 to run o3 in "high efficiency" mode against the 400 public ARC-AGI       │
│ puzzles for a score of 82.8%, and an undisclosed amount of money to run the "low efficiency" mode model to      │
│ score 91.5%. A note says:                                                                                       │
│                                                                                                                 │
│ o3 high-compute costs not available as pricing and feature availability is still TBD. The amount of compute was │
│ roughly 172x the low-compute configuration.                                                                     │
│                                                                                                                 │
│ So we can get a ballpark estimate here in that 172 * $6,677 = $1,148,444!                                       │
│ Here's how François explains the likely mechanisms behind o3, which reminds me of how a brute-force chess       │
│ computer might work.                                                                                            │
│                                                                                                                 │
│ For now, we can only speculate about the exact specifics of how o3 works. But o3's core mechanism appears to be │
│ natural language program search and execution within token space – at test time, the model searches over the    │
│ space of possible Chains of Thought (CoTs) describing the steps required to solve the task, in a fashion        │
│ perhaps not too dissimilar to AlphaZero-style Monte-Carlo tree search. In the case of o3, the search is         │
│ presumably guided by some kind of evaluator model. To note, Demis Hassabis hinted back in a June 2023 interview │
│ that DeepMind had been researching this very idea – this line of work has been a long time coming.              │
│ So while single-generation LLMs struggle with novelty, o3 overcomes this by generating and executing its own    │
│ programs, where the program itself (the CoT) becomes the artifact of knowledge recombination. Although this is  │
│ not the only viable approach to test-time knowledge recombination (you could also do test-time training, or     │
│ search in latent space), it represents the current state-of-the-art as per these new ARC-AGI numbers.           │
│ Effectively, o3 represents a form of deep learning-guided program search. The model does test-time search over  │
│ a space of "programs" (in this case, natural language programs – the space of CoTs that describe the steps to   │
│ solve the task at hand), guided by a deep learning prior (the base LLM). The reason why solving a single        │
│ ARC-AGI task can end up taking up tens of millions of tokens and cost thousands of dollars is because this      │
│ search process has to explore an enormous number of paths through program space – including backtracking.       │
│                                                       

╭─────────────────────────────────────────── World news | The Guardian ───────────────────────────────────────────╮
│ Title: International court rules against El Salvador in key abortion rights case                                │
│ Date: 2024-12-20                                                                                                │
│                                                                                                                 │
│ Court finds Central American country violated rights of a pregnant woman who was denied an abortion in 2013The  │
│ Inter-American Court of Human Rights (IACHR) has ruled that El Salvador violated the human rights of a          │
│ Salvadoran woman who was denied an abortion despite her high-risk pregnancy in 2013.The court has ordered the   │
│ Central American country to adopt “all necessary regulatory measures” so that doctors are authorised to         │
│ terminate “pregnancies that pose a risk to the woman’s life and health”. Continue reading...                    │
│                                                                                                                 │
╰──────────────── ]8;id=777261;https://www.theguardian.com/world/2024/dec/20/el-salvador-abortion-death-ruling\https://www.theguardian.com/world/2024/dec/20/el-salvador-abortion-death-ruling]8;;\ ────────────────╯

╭─────────────────────────────────────────── World news | The Guardian ───────────────────────────────────────────╮
│ Title: Could the spending bill fiasco spell the end of Trump’s hold on Republicans?                             │
│ Date: 2024-12-20                                                                                                │
│                                                                                                                 │
│ President-elect, yet to take office, faces internal rebellion, with the specter of ‘President Musk’ looming     │
│ largeDonald Trump is still a month away from returning to the White House and already his relationship with     │
│ Republicans on Capitol Hill is fraying, signalling trouble ahead for both sides.The president-elect’s inability │
│ to intimidate members of his own party in the House to back a spending resolution just to keep the government   │
│ open ahead of a midnight shutdown surely has implications for his ability to drive through his ambitious agenda │
│ in the face of tiny majorities in both congressional chambers once he returns to office. Continue reading...    │
│                                                                                                                 │
╰──────── ]8;id=987194;https://www.theguardian.com/us-news/2024/dec/20/spending-bill-trump-influence-republican-party\https://www.theguardian.com/us-news/2024/dec/20/spending-bill-trump-influence-republican-party]8;;\ ─────────╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: Party City to close all stores in the U.S. but Canadian locations to remain open                         │
│ Date: 2024-12-20                                                                                                │
│                                                                                                                 │
│ In Canada, the brand is owned by Canadian Tire who said the stores are operational.                             │
│                                                                                                                 │
╰─ ]8;id=735035;https://www.thestar.com/business/party-city-to-close-all-stores-in-the-u-s-but-canadian-locations-to-remain/article_cfbebb12-bf11-11ef-b43a-2319f712eec9.html\https://www.thestar.com/business/party-city-to-close-all-stores-in-the-u-s-but-canadian-locations-to-remain/ar]8;;\─╯

## Search Entries

In [ ]:
r.search("rate", mark_as_read=False)

╭─────────────────────────────────────────────── Huggingface Blog ────────────────────────────────────────────────╮
│ Title: The 5 Most Under-Rated Tools on Hugging Face                                                             │
│ Date: 2024-08-22                                                                                                │
│                                                                                                                 │
│ None                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────── ]8;id=920969;https://huggingface.co/blog/unsung-heroes\https://huggingface.co/blog/unsung-heroes]8;;\ ───────────────────────────────────╯

╭──────────────────────────────────────────────── CBC | Business ─────────────────────────────────────────────────╮
│ ❗Title: U.S. Federal Reserve cuts interest rates by another quarter point                                      │
│ Date: 2024-12-18                                                                                                │
│                                                                                                                 │
│ The Federal Reserve cut its key interest rate Wednesday by a quarter-point — its third cut this year — but also │
│ signalled that it expects to reduce rates next year more slowly than previously thought, with inflation still   │
│ well above the central bank's two per cent target.                                                              │
│                                                                                                                 │
╰─────────────────── ]8;id=366295;https://www.cbc.ca/news/business/fed-cuts-rates-dec-18-1.7414148?cmp=rss\https://www.cbc.ca/news/business/fed-cuts-rates-dec-18-1.7414148?cmp=rss]8;;\ ────────────────────╯

╭──────────────────────────────────────────────── CBC | Business ─────────────────────────────────────────────────╮
│ Title: Canada's inflation rate edges down to 1.9%                                                               │
│ Date: 2024-12-17                                                                                                │
│                                                                                                                 │
│ Canada's annual inflation rate ticked down to 1.9 per cent in November, with the slowdown in price growth       │
│ mostly driven by lower mortgage interest costs and cheaper travel tours, Statistics Canada said on Tuesday.     │
│                                                                                                                 │
╰──────────────── ]8;id=859870;https://www.cbc.ca/news/business/inflation-rate-november-2024-1.7412342?cmp=rss\https://www.cbc.ca/news/business/inflation-rate-november-2024-1.7412342?cmp=rss]8;;\ ────────────────╯

╭──────────────────────────────────────────────── CBC | Business ─────────────────────────────────────────────────╮
│ ❗Title: Canadian annual home sales surged in November, with prices rising as interest rates fall               │
│ Date: 2024-12-16                                                                                                │
│                                                                                                                 │
│ The Canadian Real Estate Association says the number of homes sold in November rose 26 per cent compared with a │
│ year ago, marking the second straight month of large year-over-year gains.                                      │
│                                                                                                                 │
╰───────────── ]8;id=747938;https://www.cbc.ca/news/business/crea-national-home-sales-nov-2024-1.7411557?cmp=rss\https://www.cbc.ca/news/business/crea-national-home-sales-nov-2024-1.7411557?cmp=rss]8;;\ ──────────────╯

╭───────────────────────────────────────── Toronto Star | Investigations ─────────────────────────────────────────╮
│ ❗Title: Harold the Mortgage Closer accused of charging ‘vulnerable’ people interest rates up to 76 per cent    │
│ Date: 2024-05-17                                                                                                │
│                                                                                                                 │
│ Gerstel ‘is not suitable to be licensed as a mortgage broker,’ says the province’s financial services regulator │
│ in a proposal that outlines five new cases against the GTA businessman.                                         │
│                                                                                                                 │
╰─ ]8;id=978306;https://www.thestar.com/news/investigations/harold-the-mortgage-closer-accused-of-charging-vulnerable-people-interest-rates-up-to-76-per/article_0d3c6388-13b7-11ef-866d-e36a5e8e4903.html\https://www.thestar.com/news/investigations/harold-the-mortgage-closer-accused-of-charging-vulnerable-people-i]8;;\─╯

╭──────────────────────────────────────────── Science | The Guardian ─────────────────────────────────────────────╮
│ Title: Unusual scales on crocodile heads due to skin growth rate, scientists say                                │
│ Date: 2024-12-11                                                                                                │
│                                                                                                                 │
│ Researchers find that varying stiffness and speeds at which skin grows lead to ‘mechanical’ formation of inward │
│ foldsIt sounds like a conundrum that Rudyard Kipling would have explored in his Just So Stories, but            │
│ researchers say they have the answer to how crocodiles get the scales on their heads.Many animals, from turtles │
│ to birds, have scales – hard plate-like structures that form on the skin. Continue reading...                   │
│                                                                                                                 │
╰────── ]8;id=622142;https://www.theguardian.com/science/2024/dec/11/scales-crocodile-heads-skin-growth-rate-scientists\https://www.theguardian.com/science/2024/dec/11/scales-crocodile-heads-skin-growth-rate-scientists]8;;\ ───────╯

╭───────────────────────────────────────────── Canada | The Guardian ─────────────────────────────────────────────╮
│ Title: Canada’s rate of medically assisted deaths rises to record high                                          │
│ Date: 2024-12-12                                                                                                │
│                                                                                                                 │
│ Roughly 96% of deaths by euthanasia in 2023 were for those with a terminal condition, as growth in overall      │
│ cases slowsA growing share of deaths in Canada are from euthanasia, but the vast majority are for terminal      │
│ illnesses, according to new government figures.More than 15,000 people received medical assistance in dying in  │
│ Canada in 2023, the highest figure on record. But federal statistics show the growth in cases has slowed        │
│ significantly, with assisted death making up 4.7% of deaths, compared to 4.1% the previous year. Continue       │
│ reading...                                                                                                      │
│                                                                                                                 │
╰───────────────── ]8;id=65117;https://www.theguardian.com/world/2024/dec/12/canada-medically-assisted-death\https://www.theguardian.com/world/2024/dec/12/canada-medically-assisted-death]8;;\ ─────────────────╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: Loonie dips below 70 cents U.S. for first time since start of pandemic as Freeland resignation shakes    │
│ markets                                                                                                         │
│ Date: 2024-12-18                                                                                                │
│                                                                                                                 │
│ Wednesday afternoon when the U.S. Federal reserve announced it would likely only be cutting rates twice next    │
│ year, it took another big hit.                                                                                  │
│                                                                                                                 │
╰─ ]8;id=402029;https://www.thestar.com/business/loonie-dips-below-70-cents-u-s-for-first-time-since-start-of-pandemic-as/article_1505c172-bd53-11ef-83fd-b3fe997a6769.html\https://www.thestar.com/business/loonie-dips-below-70-cents-u-s-for-first-time-since-start-of-pandemic-as/arti]8;;\─╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: Toronto area new home sales down 55% in November — with condos taking the hardest hit                    │
│ Date: 2024-12-19                                                                                                │
│                                                                                                                 │
│ New condo sales continue to suffer, selling at a rate 81 per cent lower than last November and 90 per cent      │
│ lower than the Toronto region's 10-year average.                                                                │
│                                                                                                                 │
╰─ ]8;id=936826;https://www.thestar.com/real-estate/toronto-area-new-home-sales-down-55-in-november-with-condos-taking-the-hardest-hit/article_81d16074-bd5b-11ef-b635-0fb20263dd05.html\https://www.thestar.com/real-estate/toronto-area-new-home-sales-down-55-in-november-with-condos-taking-the-har]8;;\─╯

╭──────────────────────────────────────────── Science | The Guardian ─────────────────────────────────────────────╮
│ Title: Is our model of the universe wrong? – podcast                                                            │
│ Date: 2024-12-17                                                                                                │
│                                                                                                                 │
│ For the past 10 years cosmologists have been left scratching their heads over why two methods for measuring the │
│ universe’s rate of expansion provide totally different results. There are two possible solutions to the puzzle, │
│ known as the Hubble tension: either something is wrong with the measurements or something is wrong with our     │
│ model of the universe. It was hoped that observations from the James Webb space telescope might shed some light │
│ on the problem, but instead results published last week have continued to muddy the waters. To understand why   │
│ the expansion rate of the universe remains a mystery, and what might be needed to finally pin it down,          │
│ Madeleine Finlay speaks to Catherine Heymans, the astronomer royal for Scotland and a professor of astrophysics │
│ at the University of EdinburghThe Hubble constant: a mystery that keeps getting biggerSupport the Guardian:     │
│ theguardian.com/sciencepod Continue reading...                                                                  │
│                                                                                                                 │
╰─────── ]8;id=527434;https://www.theguardian.com/science/audio/2024/dec/17/is-our-model-of-the-universe-wrong-podcast\https://www.theguardian.com/science/audio/2024/dec/17/is-our-model-of-the-universe-wrong-podcast]8;;\ ────────╯

## Tag Entry

In [ ]:
test = list(r._reader.get_entries())

In [ ]:
r._reader.set_tag(test[0], "foobar")

In [ ]:
list(r._reader.get_tags(test[0]))

[('foobar', None)]

In [ ]:
# We added a method to just return the tag key:
r._get_tags(test[0])

['foobar']

## Remove tag from entry

In [ ]:
r._reader.delete_tag(test[0], "foobar")

In [ ]:
list(r._get_tags(test[0]))

[]

## Mark as Important/Unimportant

In [ ]:
r._mark_important(test[0].feed_url, test[0].id)

In [ ]:
r._mark_unimportant(test[0].feed_url, test[0].id)

## Mark as Read / Unread

In [ ]:
r._reader.mark_entry_as_read(test[0])
r._reader.mark_entry_as_unread(test[0])

## Automatically mark entries as read

When updating feeds, entries with titles matching these strings will automatically be marked as read

In [ ]:
for i in [
    "Musk",
    "Apple",
    "Bezos",
    "Google",
    "Samsung",
    "iPhone",
    "iPad",
    "guardian.com/sport/",
    "guardian.com/football/",
    "thestar.com/sports",
]:
    r.blacklist_add(i)

In [ ]:
r.blacklist_show()

[
    'Musk',
    'Apple',
    'Bezos',
    'Google',
    'Samsung',
    'iPhone',
    'iPad',
    'guardian.com/sport/',
    'guardian.com/football/',
    'thestar.com/sports',
    'cbc.ca/sports'
]

In [ ]:
# r.update()

## Automatically mark entries as important

In [ ]:
for i in ["interest rate", "Bank of Canada", "housing market"]:
    r.whitelist_add(i)

In [ ]:
r.whitelist_show()

['interest rate', 'Bank of Canada', 'housing market']

In [ ]:
# r.update()

## Low level access to data

In [ ]:
entries = r._get_entries(limit=1)
entry = [i for i in entries][0]
entry

Entry(id='1.7416928', updated=None, title='Feds announce forum on antisemitism following incidents at Jewish school and synagogue', link='https://www.cbc.ca/news/politics/feds-announce-forum-on-antisemitism-following-incidents-at-jewish-school-and-synagogue-1.7416928?cmp=rss', author=None, published=datetime.datetime(2024, 12, 20, 22, 28, 50, tzinfo=datetime.timezone.utc), summary='<img alt="A woman in a winter coat speaks at a podium as two people stand behind her." height="349" src="https://i.cbc.ca/1.7416687.1734724108!/fileImage/httpImage/image.JPG_gen/derivatives/16x9_620/cabinet-shuffle-20241220.JPG" title="Minister of Official Languages and Associate Minister of Public Safety Rachel Bendayan speaks to reporters following a cabinet shuffle at Rideau Hall in Ottawa, on Friday, Dec. 20, 2024." width="620" /><p>The federal government will hold a national forum on combatting antisemitism in February — news that comes as police investigate a suspicious fire at a Montreal synagogue and

## CLI

In [ ]:
# | export


def main():
    fire.Fire(PyNewsReader)

In [ ]:
# | export


# if __name__ == "__main__":
#     main()

In [ ]:
# | hide

# To Do:
# * User interface

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()